In [15]:
import pandas as pd

import spotify
import utils

sp = spotify.login(scope='playlist-modify-private')
user = sp.current_user()
displayName = user['display_name']
# print(sp.recommendation_genre_seeds()['genres'])
sp.recommendation_genre_seeds()['genres']

['acoustic',
 'afrobeat',
 'alt-rock',
 'alternative',
 'ambient',
 'anime',
 'black-metal',
 'bluegrass',
 'blues',
 'bossanova',
 'brazil',
 'breakbeat',
 'british',
 'cantopop',
 'chicago-house',
 'children',
 'chill',
 'classical',
 'club',
 'comedy',
 'country',
 'dance',
 'dancehall',
 'death-metal',
 'deep-house',
 'detroit-techno',
 'disco',
 'disney',
 'drum-and-bass',
 'dub',
 'dubstep',
 'edm',
 'electro',
 'electronic',
 'emo',
 'folk',
 'forro',
 'french',
 'funk',
 'garage',
 'german',
 'gospel',
 'goth',
 'grindcore',
 'groove',
 'grunge',
 'guitar',
 'happy',
 'hard-rock',
 'hardcore',
 'hardstyle',
 'heavy-metal',
 'hip-hop',
 'holidays',
 'honky-tonk',
 'house',
 'idm',
 'indian',
 'indie',
 'indie-pop',
 'industrial',
 'iranian',
 'j-dance',
 'j-idol',
 'j-pop',
 'j-rock',
 'jazz',
 'k-pop',
 'kids',
 'latin',
 'latino',
 'malay',
 'mandopop',
 'metal',
 'metal-misc',
 'metalcore',
 'minimal-techno',
 'movies',
 'mpb',
 'new-age',
 'new-release',
 'opera',
 'pagode',

In [16]:
preferred_genres = ['afrobeat', 'alt-rock', 'alternative','bluegrass', 'blues', 'bossanova', 
                    'dancehall', 'deep-house', 'disco', 'drum-and-bass', 'dub', 'dubstep', 
                    'funk', 'groove', 'hard-rock', 'house', 'indie', 'indie-pop', 'jazz', 'punk-rock', 
                    'reggae', 'rock', 'rock-n-roll', 'rockabilly', 'ska', 'soul', 'trip-hop']

In [17]:
# target_bpm = 110.0
# bpm_bandwidth = 10.0
min_tempo = 120.0
max_tempo = 130.0
query = {
    'seed_genres': ['jazz'],
    'limit': 20,
    'country': 'CH',
}
track_attributes = {
#     'min_tempo': target_bpm - (bpm_bandwidth / 2),
#     'max_tempo': target_bpm + (bpm_bandwidth / 2),
    'min_tempo': min_tempo,
    'max_tempo': max_tempo,
    'min_danceability': 0.66,
    'max_liveness': 0.33,
    'max_speechiness': 0.33,
    # 'max_instrumentalness': 0.33,
    'max_acousticness': 0.33,
    'target_time_signature': 4,
    # 'min_valence': 0.5,
    'min_popularity': 1,
}
response = sp.recommendations(**query, **track_attributes)
tracks = response['tracks']
len(tracks)

0

In [6]:
response.keys()

dict_keys(['tracks', 'seeds'])

In [7]:
response['seeds']

[{'initialPoolSize': 767,
  'afterFilteringSize': 4,
  'afterRelinkingSize': 4,
  'id': 'jazz',
  'type': 'GENRE',
  'href': None}]

In [8]:
tracklist = []
for track in tracks:
    tracklist.append({
        'id': track['id'],
        'name': track['name'],
        'artist_id': track['artists'][0]['id'],
        'artist_name': track['artists'][0]['name'],
        'popularity': track['popularity'],
    })
df = pd.DataFrame(tracklist)

In [9]:
df

,id,name,artist_id,artist_name,popularity
0,1CDBaGlisZlOJzvx88lL8A,Song For My Father,5ZATfKurLqflrBhv2FLht5,Horace Silver,58
1,6HRUw67PN9Q8q4y2FDQOU6,Hot Mallets - 2002 Remastered,2PjgZkwAEk7UTin4jP6HLP,Lionel Hampton,26
2,0KeQYOvK25PpIXJLzxcy2u,"Straight, No Chaser (Take 3)",4PDpGtF16XpqvXxsrFwQnN,Thelonious Monk,10
3,5H4bQ14RjOVM4eKtaIpn3B,Clarinet Marmalade,3kfbYBrL3MCPAMUwKDRpc7,Bix Beiderbecke,33


In [10]:
df

,id,name,artist_id,artist_name,popularity
0,1CDBaGlisZlOJzvx88lL8A,Song For My Father,5ZATfKurLqflrBhv2FLht5,Horace Silver,58
1,6HRUw67PN9Q8q4y2FDQOU6,Hot Mallets - 2002 Remastered,2PjgZkwAEk7UTin4jP6HLP,Lionel Hampton,26
2,0KeQYOvK25PpIXJLzxcy2u,"Straight, No Chaser (Take 3)",4PDpGtF16XpqvXxsrFwQnN,Thelonious Monk,10
3,5H4bQ14RjOVM4eKtaIpn3B,Clarinet Marmalade,3kfbYBrL3MCPAMUwKDRpc7,Bix Beiderbecke,33


In [11]:
df_sorted = df.sort_values('tempo')
df['cum_mins'] = (df_sorted['duration_ms'].cumsum()) / 1000 / 60
df[['id', 'artist_name', 'name', 'tempo', 'duration_ms', 'cum_mins']]

KeyError: 'tempo'

In [10]:
playlist_name = (str(min_tempo) + "-" + str(max_tempo) + " BPM " + ', '.join(query['seed_genres']).title())
playlist_name

'110.0-115.0 BPM Deep-House, House, Disco, Dance, Funk'

In [26]:
playlist = sp.user_playlist_create(
    user['id'], 
    playlist_name, 
    public=False, 
    description="Created with BPMPlaylist.py"
)
sp.user_playlist_add_tracks(user['id'], playlist['id'], list(df['id']))
sp.user_playlist(user['id'], playlist['id'], fields="tracks")['tracks']['total']

20